#### Algorithm. 
_Man- did I take some really long breaks before and after each Level 4 problems! Maybe I was getting bored (even though the questions got really interesting), or maybe I'm just worried about my research (life of a PhD student- eh!)_

This one is a mixed bag. I initially thought it's gonna take me some time- but ultimately the base code took only a few hours. Basically, the problem asks us to pair up a number of guards with different amounts of bananas in thumb wrestling games where the guard with less bananas will bet all of their bananas (and will win due to "overconfident guard psychology"). This will continue for a number of rounds till both guards have the same amount of bananas (which is why we never start with a guard pair with same number of bananas). The goal is to keep the guards distracted as long as possible- so we want to pair them up in such a way that their games continue back and forth in an infinite loop.

The first part of the problem is to detect the pairs that will result in infinite loops. Once that is done, we can make a <code>list</code> with individual loop counts for all guards as well as a <code>list</code> for the loop partners for each guard to use in the second part. Now, we can detect an infinite loop in various ways - 
* We can let the guards play a certain number of rounds and once the game continues past that round- we'll call it an infinite loop. This is the naive approach and we arbitrarily decide on the terminating round. I started with this using <code>round = 10</code>. And, to my utter surprise, **this worked even for all the test examples with sufficient rounds** (*i.e.*, <code>round = 30</code>). 
* We can keep track of the banana pairs already seen in the game and if any of them repeats- it'll imply that they're in an infinite loop. A bit better than the naive approach- but heavily dependent upon the number of bananas at play. I tried this second with a hypothetical case of banana pairs $(2^{30},\ 2^{30} - 2)$ which took around 30 rounds to get the first repeatation. Note that, I consider the pair $(x, y)$ same as the pair $(y, x)$ since in both cases the game is going in the same direction (as the total banana count _never_ changes) with the player roles reversed. 
* We can check the **greatest common divisor (GCD)** of the pair $(x, y)$. From the [Euclidean algorithm](https://en.wikipedia.org/wiki/Euclidean_algorithm) for calculating GCD- it can be implied that the pair that has the same GCD throughout a number of rounds will go on forever, while for a terminating pair, it will be doubled in each round. We might also have a composite situation where the first few rounds will be nonrepeating and GCD will increase, but once the game enters a loop, the GCD will get fixed. So, intuitively, I kept track of the GCDs of each pair in each round (again a <code>list</code>) and whenever a certain GCD appears more than a few times (used 3 as tolerance), I infer it to be an infinite loop. Have to thank internet again for this one- somebody on stack overflow mentioned that the GCD for a looping pair never changes- I got intrigued and checked it out to be true! The rest got easy after that. This is what I used in the final version of my code since this doesn't need to be run as long as the other two. 

The second part is to pair up the guards so that a maximum possible number of guards will be busy. So, this is the problem of choosing the best pairs that maximizes the number of infinite loops. I tried two simple things here- and surprisingly, one of them worked! 
* We can sort the guards ascendingly according to their number of infinite game pairings. Then, choose the guard with the lowest number of loops and pair him/her up with the partner who has the lowest number of loops among all the partners who can have infinite loop games with the  guard (and are not occupied in another game). This intuitively makes sense since if we want the best possible combo of guards to maximize loops- we must start with the guard who has the lowest number of loops with the other guards. And we can think of similar logic for his/her partner too. But the caveat is this might cause us to lose some other possible pairings that will maximize loops. I drew up some random examples (pen and paper)- tried with this and faced this issue a couple of times. 
* We still sort the guards according to the number of infinite loops and choose the guard with the lowest number of loops. But, this time we pair him/her up with the partner who has the most number of infinite loop partners among all available partners. My rationale for this is that we still have to start with the guard with the lowest number of loops but we should choose a partner who is less restrictive- and thus not blocking some passible pairings (like we did when we chose the partner with lowest loops) and potentially keeping a lot of other loop pairings unoccupied. This is what I went with at the end- and it worked for all cases without any special tweaking! 

So, overall this was nice combinatorial problem that was fun to solve and easier than it appears to be at first (at least for me)! 


In [2]:
""" Challenge 4.2. Loop maximization problem """

import numpy as np, itertools as it

def thumb_wrestling(pair, progress = False):
    ## Simulates the thumb wrestling game between two guards- returns True if the game 
    ## ends up in a loop, False otherwise
    if progress:    print("Start: banana pair = ", pair),
    
    ## If the sum is odd, the participants will never have equal number of bananas... 
    if sum(pair) % 2:
        if progress:    print("Seems like this game will go on forever!")
        return True
    
    ## If the sum is even, the game can stop at one point... 
    rounds, GCD = 0, [np.gcd(*pair)]
    while pair[0] != pair[1]:
        rounds += 1;    bet, win = min(pair), int(pair[0] < pair[1])
        pair = (pair[0] - (-1)**win * bet, pair[1] + (-1)**win * bet)
        GCD.append(np.gcd(*pair))
        if progress:    print("Round %d: banana pair = %s, \tGCD = %d" % (rounds, str(pair), GCD[-1]))

        ## For a looping game, the GCD of the two numbers will become fixed after a few rounds- 
        ## while for a terminating game, it will be doubled in each round... 
        if sum(GCD[-1] == np.array(GCD)) > 3:
            if progress:    print("Seems like this game will go on forever!")
            return True
    ## End of loop ##
    
    if progress:    print("Game over! Total rounds played =", rounds)
    return False

def solution(banana_list):
    guards = len(banana_list)
    
    ## Count loops for each guard & store loop partners... 
    loops, partners = [0 for _ in range(guards)], [[ ] for _ in range(guards)]
    for i, j in it.combinations(range(guards), 2):
        if thumb_wrestling((banana_list[i], banana_list[j]), progress = False):
            loops[i] += 1;    partners[i].append(j)
            loops[j] += 1;    partners[j].append(i)
    sorted_guards = sorted(range(guards), key = lambda gg: loops[gg])    # Sort by loop counts
    
    ## Get best pairings & count occupied guards... 
    ##  Start with the guard with lowest loop count, pair him/her with the partner 
    ##  with highest loop count from available partners (not in another game already) 
    pairings, occupied = [ ], [ ]
    for guard in sorted_guards:
        if len(occupied) == guards:    break
        if guard not in occupied:
            available = [pp for pp in partners[guard] if pp not in occupied]
            if len(available):
                if len(available) > 1:
                    available = sorted(available, key = lambda pp: -loops[pp])
                pairings.append((guard, available[0]))
                occupied += list(pairings[-1])
    ## End of loop ##
    
    left = guards - len(occupied)
    return left


In [3]:
## Examples... 
%time print("Number of guards left to watch the prisoners = ", solution(banana_list = [1, 1]))
%time print("Number of guards left to watch the prisoners = ", solution(banana_list = [1, 7, 3, 21, 13, 19]))
%time print("Number of guards left to watch the prisoners = ", solution(banana_list = [1, 7, 5, 3, 19, 13, 53, 61]))
%time print("Number of guards left to watch the prisoners = ", solution(banana_list = [1, 3, 3, 1, 5, 1]))
%time print("Number of guards left to watch the prisoners = ", solution(banana_list = [2**30 - 1, 2**30 - 3]))

Number of guards left to watch the prisoners =  2
Wall time: 0 ns
Number of guards left to watch the prisoners =  0
Wall time: 1.99 ms
Number of guards left to watch the prisoners =  0
Wall time: 1.99 ms
Number of guards left to watch the prisoners =  4
Wall time: 0 ns
Number of guards left to watch the prisoners =  0
Wall time: 0 ns
